## 보스턴 주택 가격 데이터셋

1970년 중반 보스턴 외곽 지역의 범죄율, 지방세율 등의 데이터가 주어졌을 때 주택 가격의 중간 값을 예측해 보겠습니다.

여기서 사용할 데이터셋은 이전 두 개의 예제와 다릅니다. 데이터 포인트가 506개로 비교적 개수가 적고 404개는 훈련 샘플로 102개는 테스트 샘플로 나누어져 있습니다. 입력 데이터에 있는 각 특성(예를 들어 범죄율)은 스케일이 서로 다릅니다. 어떤 값은 0과 1 사이의 비율을 나타내고 어떤 것은 1과 12 사이의 값을 가지거나 1과 100 사이의 값을 가집니다.


In [ ]:
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [ ]:
print(train_data.shape)
test_data.shape

(404, 13)


(102, 13)

In [ ]:
#데이터 준비
#테스트 데이터를 정규화 
mean = train_data.mean(axis=0)
train_data -= mean 
std = train_data.std(axis=0)
train_data/=std

test_data -= mean
test_data /= std

In [ ]:
train_data

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [ ]:
#모델 구성
from tensorflow.keras import models,layers

def bulid_model():
  model = models.Sequential()
  model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64,activation='relu'))
  #주택 가격을 에측하는 거니깐 활성화함수 안 주고 그냥 바로 출력하게끔 
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
  return model

In [ ]:
import numpy as np

k=4
# 몫 구하기
num_val_samples = len(train_data)// k
num_epochs=100
all_scores= []
for i in range(k):
  #검증 데이터 준비 : K번째 분할 
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    #훈련 데이터 준비 : 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0)
    #케라스 모델 구성 
    model = bulid_model()
    model.fit(partial_train_data,partial_train_targets,
              epochs = num_epochs,batch_size=1,verbose=0)
    # 검증 세트로 모델 평가
    val_mse,val_mae = model.evaluate(val_data,val_target,verbose=0)
    all_scores.append(val_mae)

In [ ]:
all_scores

[2.1657700538635254, 2.223945379257202, 2.901470899581909, 2.3033413887023926]

In [ ]:
np.mean(all_scores)

2.3986319303512573

In [ ]:
from tensorflow.keras import backend as K
# 메모리 해제
K.clear_session()

In [ ]:
num_epochs=500
all_mae_histories= []
for i in range(k):
  #검증 데이터 준비 : K번째 분할 
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    #훈련 데이터 준비 : 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0)
    #케라스 모델 구성 
    model = bulid_model()
    history = model.fit(partial_train_data,partial_train_targets,
             validation_data = (val_data,val_target),
             epochs = num_epochs, batch_size=1, verbose=0)
    # 검증 세트로 모델 평가
    val_mse,val_mae = model.evaluate(val_data,val_target,verbose=0)
    all_scores.append(val_mae)

In [ ]:
all_scores

[2.1657700538635254,
 2.223945379257202,
 2.901470899581909,
 2.3033413887023926,
 2.3225696086883545,
 3.043163537979126,
 2.5727546215057373,
 3.1763737201690674]

In [ ]:
# 과대적합 때문인지 epochs 100 보다 오차가 크다
np.mean(all_scores)

2.5886736512184143

In [ ]:
#모델 구성
from tensorflow.keras import models,layers
import tensorflow as tf
def bulid_model():
  model = models.Sequential()
  model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64,activation='relu'))
  #주택 가격을 에측하는 거니깐 활성화함수 안 주고 그냥 바로 출력하게끔 
  model.add(layers.Dense(1))
  # 버전차이가 있어서 에러가 발생 메트릭스를 바꿔줘야함
  model.compile(optimizer='rmsprop', loss='mse', metrics=[tf.keras.metrics.MeanAbsoluteError()])
  return model

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]